In [1]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy.dialects.postgresql import JSON, JSONB
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import json
import csv
import datetime
from dateutil.parser import parser
import os
from pprint import pprint
from pathlib import Path
import glob
#from odo import odo
import importlib
import itertools
import sklearn.cluster
import pandas as pd
import numpy as np
import math
import operator
import sklearn
import matplotlib.pyplot as plt

#importlib.invalidate_caches()
%load_ext autoreload
%autoreload 2

#### Reading in Audio Features

In [2]:
Compare = pd.read_pickle('../Pickle_Files/tracks.pkl')
audio_features = pd.read_pickle('../Pickle_Files/audio_features.pkl')
track_elems = pd.read_pickle('../Pickle_Files/track_elems.pkl')

#### Drops all uncommon songs

In [3]:
Compare = pd.merge(audio_features[['track_uri']],Compare,on='track_uri')

#### Ordering tracks

In [4]:
Compare = Compare.sort_values('track_uri',ascending=True)

In [5]:
Compare = Compare.reset_index().drop(['index'],axis=1)

In [6]:
len(Compare)

2254670

#### Creating Dataframe 'Cosine'

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
cosine = pd.read_pickle('../Pickle_Files/cosine_dataframe.pkl')

In [9]:
X = cosine.values

#### Normalizing values before cosine calculated

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit(cosine.values).transform(cosine.values)

# Recommender System
<br>-  This Recommender System takes in an input dataframe and makes a recommendation using the cosine similarity of the provided features.
<br>-  It chooses N recommendations of the length of the training set and uses ".sample()" to pick a random number of recommendations the same length as the test set.
<br>-  It then calculates the precision and recall of the test set.

The definition below calculates the precision and recall of a given recommender system

In [11]:
def recommender_precision(predicted, actual):
    def calc_precision(predicted, actual):
        prec = [value for value in predicted if value in actual]
        prec = np.round(float(len(prec)) / float(len(predicted)), 4)
        return prec

    precision = np.mean(list(map(calc_precision, predicted, actual)))
    return precision


def recommender_recall(predicted, actual):
    def calc_recall(predicted, actual):
        reca = [value for value in predicted if value in actual]
        reca = np.round(float(len(reca)) / float(len(actual)), 4)
        return reca

    recall = np.mean(list(map(calc_recall, predicted, actual)))
    return recall

#### Creating Two Empty Lists, Precision and Recall
<br> These are used in the Recommender System for calculations

In [12]:
Precision = []
Recall = []

In [13]:
def Recommender_System(df):
    
    # Obtaining index values to later compare 
    
    df = pd.merge(df,Compare['track_uri'].reset_index(),on='track_uri')
    
    #df = df.set_index('index')

    #df = df.reset_index()

    df = df[['track_uri','index']]

    df = pd.merge(cosine,df,on='track_uri')

    df = df.set_index('index')

    df = df.drop('track_uri',axis=1)

    df_values = df.index.values

    # Creates training and test set of .7 and .3

    msk = np.random.rand(len(df)) < 0.7

    dftrain = df[msk]

    dftest = df[~msk]

    dfvalues = dftrain.index.values # Index values used in cosine similarity
    
    num_tracks_desired = 1 ## Recommends only one song
    def rec_top_n(n):
        return np.argsort(cosine_similarity(X, X[dfvalues]), axis=0)[::-1][1:n] # Cosine similarity finds closest to each song
    # Prints index of all recommended songs
    print(rec_top_n(num_tracks_desired+1))
    test = rec_top_n(num_tracks_desired+1).flatten()

    test = Compare.iloc[test]

    testset = Compare[['track_name','track_uri','artist_name']].iloc[dftest.index]
    
    prediction = test.sample(len(dftest))
    prediction = prediction[['track_name','track_uri','artist_name']]
    prediction.columns = ['track_name_pred', 'track_uri_pred','artist_name_pred']

    # Evalution

    pred = [prediction.index.values]
    # Uses index to check predicted against actual
    actual = [dftest.index.values] 

    #returns Precision 
    print('\nPrecision: ' +str(recommender_precision(pred,actual)))
    Precision.append(recommender_precision(pred,actual))
    #returns Recall
    print('\nRecall: ' +str(recommender_recall(pred,actual)))
    Recall.append(recommender_recall(pred,actual))
    
    # Returns songs predicted
    
    prediction = prediction.reset_index()
    testset = testset.reset_index()
    prediction = prediction.drop(['index'],axis=1)
    testset = testset.drop(['index'],axis=1)
    final = testset.join(prediction)
    return final

### Example with Popular DJ's

In [14]:
pop = pd.read_pickle('../Pickle_Files/Song_Popularity.pkl')
pop = pop[['track_uri','Popularity_Rating_Normalized']]

In [15]:
Compare2 = pd.merge(Compare,pop,on='track_uri')

In [16]:
one = Compare2[Compare2['artist_name']=='Avicii'].sort_values(
    'Popularity_Rating_Normalized' ,ascending=False).head(10) 
two = Compare2[Compare2['artist_name']=='Hardwell'].sort_values(
    'Popularity_Rating_Normalized' ,ascending=False).head(10)   
three = Compare2[Compare2['artist_uri']==('spotify:artist:2CIMQHirSU0MQqyYHq0eOx')].sort_values(
    'Popularity_Rating_Normalized' ,ascending=False).head(10)#deadmau5
four = Compare2[Compare2['artist_name']=='Swedish House Mafia'].sort_values(
    'Popularity_Rating_Normalized' ,ascending=False).head(10)   
five = Compare2[Compare2['artist_name']=='Calvin Harris'].sort_values(
    'Popularity_Rating_Normalized' ,ascending=False).head(10)  

In [17]:
cols = five.columns

all_songs = pd.concat([one.sample(5), two.sample(5),three.sample(5),four.sample(5),five.sample(5)], ignore_index=False)

In [19]:
all_songs.sample(5)

,track_uri,album_name,album_uri,artist_name,artist_uri,duration_ms,pos,track_name,seconds,minutes,Popularity_Rating_Normalized
240100,spotify:track:0pJPdSVti6cTM1Q6xYGmcf,Funk Wav Bounces Vol.1,spotify:album:2HaqChIDc5go3qxVunBDK0,Calvin Harris,spotify:artist:7CajNmpbOovFoOoasH2HaY,240093,None,Faking It,240.093,4.001550,0.659872
2250763,spotify:track:7zBjb6MPNRkeALdgGQIvou,Call Me a Spaceman,spotify:album:4XvGoskbDSjgYsSCtajnOm,Hardwell,spotify:artist:6BrvowZBreEkXzJQMpL174,189375,None,Call Me a Spaceman - Radio Edit,189.375,3.156250,0.625344
1604801,spotify:track:5XZrF0we6XyC3MXJMxXhEU,Save The World,spotify:album:7kYHK7Vr4BLSiFWauoKgxE,Swedish House Mafia,spotify:artist:1h6Cn3P4NGzXbaXidqURXs,381056,None,Save the World - Zedd Remix,381.056,6.350933,0.477268
138676,spotify:track:0TaJUvjmA2WToXM0FyUyRR,18 Months,spotify:album:4TjPTVBmdyaowp08j2toa0,Calvin Harris,spotify:artist:7CajNmpbOovFoOoasH2HaY,247933,None,Thinking About You,247.933,4.132217,0.727719
312607,spotify:track:14epucTZd8dLIYipdByNZS,Until Now,spotify:album:4ljisoNarj0BpQSMIEv88L,Swedish House Mafia,spotify:artist:1h6Cn3P4NGzXbaXidqURXs,374124,None,Antidote,374.124,6.235400,0.604462


Here we can see that the recommender has recommeded a song by Hardwell, it has also recommeded songs from other DJs like Armin van Buuren and Adventure Club

In [20]:
Recommender_System(all_songs)

[[ 618409 1043434  936600  472544  544471 1892601  450405 2147494  987937
   928783  685321 2108898 1148344]]

Precision: 0.0

Recall: 0.0


,track_name,track_uri,artist_name,track_name_pred,track_uri_pred,artist_name_pred
0,Blame,spotify:track:07nH4ifBxUB4lZcsf44Brn,Calvin Harris,Fast Life (feat. Vic Mensa),spotify:track:1Y7q2n46eW4po5GqgW18wR,Asher Roth
1,Off the Hook - Radio Edit,spotify:track:0GKyUNSSpJ2DY1Epx9fG0K,Hardwell,Without U,spotify:track:3PDNbQyXxsFnXwnM6lnnIC,Beach Goons
2,Strobe,spotify:track:0YchKPFRbVwwO4dH5hKrRi,deadmau5,Drunk On You,spotify:track:3b7CDTKB0SRTmQ6ytYi5vZ,Luke Bryan
3,Ghosts ‘n’ Stuff - NERO Remix,spotify:track:0iTzJzy5qBL4NaEDyi1HMK,deadmau5,Nightcrawler,spotify:track:3xby7fOyqmeON8jsnom0AT,Travis Scott
4,Rollin,spotify:track:0lMbuWUpfTWhEmOKxppEau,Calvin Harris,Crave You - Adventure Club Remix,spotify:track:283LAlCpfArRZOXPfkOQj0,Flight Facilities
5,How We Do - Original Mix),spotify:track:4dHLeKmHhjyu1pfzE1j9Ac,Hardwell,Lost - Explicit Album Version,spotify:track:6XIiqQp6A0gzaIEaDszkZX,Gorilla Zoe
6,One (Your Name) [feat. Pharrell],spotify:track:5EJcVJZBl6HTfSf6z4dC78,Swedish House Mafia,Young Again,spotify:track:1sDq5xK0N3DSlYv4ZrIUpS,Hardwell
7,The Days,spotify:track:5Iy2Jj87Ha0C0IBlNE1I4y,Avicii,Island In The Sun,spotify:track:2MLHyLy5z5l5YRp7momlgw,Weezer
8,Save the World - Radio Mix,spotify:track:65hRZOQkVjmBdusz1N7aOv,Swedish House Mafia,Disparate Youth,spotify:track:7d23MhPFE9eB3U8DPRirnL,Santigold
9,Without You (feat. Sandro Cavazza),spotify:track:6WbADFqMvR8N5u0BvtsWQE,Avicii,Another You - Radio Edit,spotify:track:7Hp7PPT6I4xk9svmfcMoVK,Armin van Buuren


## Precision and Recall on 50 Random Playlists

#### Removing Playlists with less than 20 songs

In [21]:
track_elems = track_elems[track_elems['COUNT(*)']>=2000]

In [22]:
remove = track_elems.groupby('pid').nunique()

In [23]:
new_track_elems = remove[remove['COUNT(*)']>20]

In [24]:
new_track_elems = new_track_elems.drop(['pid','pos','track_uri'],axis=1)

In [25]:
new_track_elems.columns = ['num_tracks']

In [26]:
new_track_elems = pd.merge(track_elems,new_track_elems.reset_index(),on='pid')

#### Picks 50 random playlists

In [27]:
mylist = new_track_elems.pid.unique()

import random

sample_size = 50
sorted_sample = [mylist[i] for i in sorted(random.sample(range(len(mylist)), sample_size))]

df_dict = {name: track_elems.loc[track_elems['pid'] == name] for name in sorted_sample}

#### Sample of one of the playlists

In [28]:
df_dict[sorted_sample[0]].head()

,pid,pos,track_uri,COUNT(*)
7199,172772,24,spotify:track:1XazlqB4YAtvouCmJPWXlM,5757
14214,172772,35,spotify:track:4ly1QBXEwYoDmje9rmEgC4,7880
99208,172772,18,spotify:track:63OQWdXMq0Pe6pRpWY5gO0,6168
124001,172772,26,spotify:track:7EAFj7eJcMF5koWSRJVZcL,4822
154315,172772,27,spotify:track:4VFE6ZNqa8jHAmbYICoAFg,5630


#### Loop that runs recommendation system for all 50 playlists

In [29]:
x = 0
while x < 50:
    Recommender_System(df_dict[sorted_sample[x]])
    x += 1

[[1698807 1631953  776263 1152792 1987829 1230325 2110605 1664819 2089364
  1813528 1305110 2023254 1528878  477679  629311  910872 2123942  980924
  1861498 2211908 1888480  399066 1748487 1152285  725553 1659525  292473
  1430932 2138411 1158785   63896 1628909 1998886 1013991 1911970   18533
  2202492 1152792 2055466  182214 1945476]]

Precision: 0.0

Recall: 0.0
[[1220815  585513 1256439 1256439 1297775 1675774 1366602   47718 1136973
   968952  493049 2246542  446308 1382765 1311301 1014504 1219536 1397747
  1983307  665004 1555207  199372 1670034 1167670  399066  923142  561814
  1368697 1651652 2234531 1349357  760959 1968641 2062035 1690679  182214
  1583026 1148506 1311986  974211   18533 1641622 2252238 1162002 1642692
  1799152  515374]]

Precision: 0.0

Recall: 0.0
[[ 837998  245210 1212392 1753223  567286 1161623 1430608 1224860 2162726
   533206  163158  301366 1694600  980924 1948221 2017947 1518177 1964013
  2244257 1192559  979403 1542252 2032389  776169  195375  51825


Precision: 0.0

Recall: 0.0
[[ 433744 2250722 1726223 1163236  666230  704900 1265726 2250722 1824143
   269840 2017947 1898666  572704 1234031 1492271  389304 1958637 1365982
   100879  890096  579565]]

Precision: 0.0

Recall: 0.0
[[1034237 2186723 1643738 1473365  221302 1152792 2159294      56 2130514
  2115903  236968 1487561 1809589  642641   17223 2049560 2233270  445370
  2204176  260649  404102 1161623  103871 1105282  967377 1922757 1271652
  1898142 1269413 1451312 1188854 2028474 1812680 1871709  385815 1008430
   244994  382818 1300446  483917 1630000 1044600  179946 1433175  769571
   573045 2246542 1514701 1555207  131560 1633209  214552  973947 2089990
    81416 1764427  787289 1448721 1520485   19512  916460 1898666  707293
  1482754  913504 1265220 1158785  121533  839920  408880  210218 1916088
   654369  705612 1652093  154593  350654  587357 1992588  718189 1952741
  1803647 1048929 1918340 1393259 1756499 2024654]]

Precision: 0.0357

Recall: 0.0357
[[1286884  24

In [30]:
print(Precision)
np_Precision = np.array(Precision)
print('\nMean Precision = '+str(np_Precision.mean())+'\n')

print(Recall)
np_Recall = np.array(Recall)
print('\nMean Recall = '+str(np_Recall.mean()))

[0.0, 0.0, 0.0, 0.0, 0.037, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0357, 0.0323, 0.0, 0.0, 0.0645, 0.0588, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Mean Precision = 0.005456862745098039

[0.0, 0.0, 0.0, 0.0, 0.037, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0357, 0.0323, 0.0, 0.0, 0.0645, 0.0588, 0.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Mean Recall = 0.005456862745098039


# Conclusion

<br>-The precision and recall has increased further when using the PCA reduced clusters showing their value to the recommender system.
<br>-The system is now strong enough to be put to the task of <b>Playlist Continuation</b>